In [7]:
import json
import spacy
from gensim.models import Word2Vec
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import load_model

import random

# Text Preprocessing

In [3]:
nlp = spacy.load("en_core_web_sm")

with open('intents.json', 'r') as f:
    data = json.load(f)

words = []
tags = []
words_and_tags = []

def preprocessing(text):
    # tokenize text
    doc = nlp(text)
    # lemmatize and lower case text
    return [token.lemma_.lower() for token in doc if not token.is_punct]

for intent in data['intents']:
    # add tag to the "tags" list if it is not already in the list
    tags.append(intent['tag']) if intent['tag'] not in tags else None

    for pattern in intent['patterns']:
        preprocessed_words_list = preprocessing(pattern)
        words.extend(preprocessed_words_list)
        words_and_tags.append((preprocessed_words_list, intent['tag']))

# remove duplicates and sort
words = sorted(set(words))
tags = sorted(tags)

print(f"Tags: {tags}")
print(f"Words: {words}")
print(f"Words_and_Tags: {words_and_tags}")


Tags: ['goodbye', 'greetings', 'intro']
Words: ['a', 'adieu', 'afternoon', 'be', 'bye', 'call', 'day', 'evening', 'farewell', 'get', 'good', 'goodbye', 'great', 'have', 'hello', 'hey', 'hi', 'how', 'howdy', 'i', 'introduce', 'later', 'leave', 'morning', 'name', 'now', 'out', 'peace', 'please', 'see', 'should', 'to', 'up', 'what', 'who', 'ya', 'yo', 'you', 'your', 'yourself']
Words_and_Tags: [(['hello'], 'greetings'), (['hi'], 'greetings'), (['hey'], 'greetings'), (['what', 'be', 'up'], 'greetings'), (['yo'], 'greetings'), (['howdy'], 'greetings'), (['good', 'morning'], 'greetings'), (['good', 'afternoon'], 'greetings'), (['good', 'evening'], 'greetings'), (['goodbye'], 'goodbye'), (['bye'], 'goodbye'), (['farewell'], 'goodbye'), (['adieu'], 'goodbye'), (['see', 'you', 'later'], 'goodbye'), (['i', 'get', 'to', 'leave', 'now'], 'goodbye'), (['have', 'a', 'great', 'day'], 'goodbye'), (['see', 'ya'], 'goodbye'), (['peace', 'out'], 'goodbye'), (['what', 'be', 'your', 'name'], 'intro'), (['i

# Convert words to numerical features using Word2Vec

In [5]:
# Create a list of sentences for gensim's Word2Vec
sentences = [each[0] for each in words_and_tags]

# Train Word2Vec model
w2vmodel = Word2Vec(sentences, vector_size=50, window=5, min_count=1, sg=1, workers=4, compute_loss=True)
w2vmodel.save("word2vec_model")

def vectorization(sentence, model, words):
    sentence_vec = np.zeros((len(words), 50))
    for word in sentence:
        if word in model.wv:
            sentence_vec += model.wv[word]
    return sentence_vec.flatten()

# Create training data
train_x = []
train_y = []
output_empty = [0] * len(tags)

for doc in words_and_tags:
    bag = vectorization(doc[0], w2vmodel, words)
    train_x.append(bag)
    output_row = list(output_empty)
    output_row[tags.index(doc[1])] = 1
    train_y.append(output_row)

train_x = np.array(train_x)
train_y = np.array(train_y)

# Build and Train Neural Network

In [6]:
# Build the model
model = Sequential()
model.add(Dense(128, input_shape=(train_x.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(tags), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)

# Save the model
model.save("chatbot_model.h5")
print("Model created and saved")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.3659 - loss: 1.0935
Epoch 2/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4961 - loss: 1.0771 
Epoch 3/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5884 - loss: 1.0664 
Epoch 4/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7714 - loss: 1.0232 
Epoch 5/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6430 - loss: 1.0000 
Epoch 6/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9266 - loss: 0.9065 
Epoch 7/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8214 - loss: 0.9067 
Epoch 8/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7519 - loss: 0.8768 
Epoch 9/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9155 - loss: 0.7701 
Epoch 10/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7019 - loss: 0.7840  
Epoch 11/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9627 - loss: 0.7011 
Epoch 12/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9661 - los

Model created and saved


In [8]:
# Load the model
model = load_model("chatbot_model.h5")

def predict_class(sentence, model, words, w2v_model):
    sentence_vector = vectorization(preprocessing(sentence), w2v_model, words)
    res = model.predict(np.array([sentence_vector]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    # Sort by probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": tags[r[0]], "probability": str(r[1])})
    return return_list

def get_response(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model, words, w2vmodel)
    res = get_response(ints, data)
    return res


In [ ]:
while True:
    message = input("You: ")
    if message.lower() == "quit":
        break
    response = chatbot_response(message)
    print(f"Bot: {response}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Bot: Hi, how can I help you?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Bot: I'm Sam - I'm an AI chatbot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Bot: My name is Sam - I'm an AI chatbot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Bot: Hello, what can I do for you?
